In [17]:
# IMPORT LIBRARIES

import numpy as np 
import sympy as sp
import matplotlib.pyplot as plt

In [18]:
# DATA 

q = 25 # Power density [W/cm3]
T = 60 # Operating core temperature [°C]
T_0 = 20 # Operating reflector temperature [°C]
nu = 2.44 # Neutrons per fission [-]
rho_H2O = 1 # Water density [g/cm3]
rho_235 = 19.1 # Uranium 235 density [g/cm3]
D_Core = 0.16 # Core diffusion coefficient [cm]
D_Re = 0.16 # Reflector diffusion coefficient [cm]
B_Core = 0.05624 # Core Buckling [1/cm]
NAV = 6.022 * 10 ** 23 # Avogadro number [atoms/mol]
MM_H2O = 18.015 # Water molar mass [g/mol]
MM_235 = 235.044 # Uranium-235 molar mass [g/mol]

conversion = 6.242 * 10 ** 18 # Conversion coefficient eV-J
E_Fission = 211.5 * 10 ** 6    # Energy released by fission [eV]
E_Fission = E_Fission/conversion # Energy released by fission [J]

In [19]:
# FUNCTION AND SYMBOLIC VARIABLES DEFINITION
x = sp.symbols('x')
B = sp.symbols('B', nonnegative = True) 
L_c = sp.symbols('L_c', nonnegative = True)
L_r = sp.symbols('L_r', nonnegative = True)
D = sp.symbols('D', nonnegative = True) # D_R = d_C
H = sp.symbols('H', nonnegative = True) # Core width
F_c = sp.Function('phi_c', real = True)(x)
F_r = sp.Function('phi_r', real = True)(x)

In [20]:
# ODE + BC

ODE_c = sp.Eq(sp.diff(sp.diff(F_c, x)) + B** 2 * F_c, 0)
print('ODE FOR THE CORE:')
display(ODE_c)

ODE_r = sp.Eq(sp.diff(sp.diff(F_r, x)) - (1/L_r**2) * F_r, 0)
print('ODE FOR THE REFLECTOR:')
display(ODE_r)

flux_c = sp.dsolve(ODE_c)
sC1 = sp.symbols('C1')   
sC2 = sp.symbols('C2')
print('GENERAL SOLUTION FOR THE FLUX IN THE CORE:')
display(flux_c)

flux_r = sp.dsolve(ODE_r)
sC3 = sp.symbols('C3')
sC4 = sp.symbols('C4')
flux_r = flux_r.subs({sC1: sC3, sC2: sC4})
print('GENERAL SOLUTION FOR THE FLUX IN THE REFLECTOR:')
display(flux_r)

# Vacuum BC
vacuum = sp.Eq(
    flux_c.rhs.subs(x, 0),
    0
)
print('VACUUM CONDITION IN THE CORE:')
display(vacuum)

# Bounded BC (da migliorare, non so farla generica...)
flux_r = flux_r.subs(sC4, 0)
print('GENERAL SOLUTION FOR THE FLUX IN THE REFLECTOR AFTER BOUNDED BC:')
display(flux_r)

# Continuity BC
continuity_f = sp.Eq(
    flux_c.rhs.subs(x, H), 
    flux_r.rhs.subs(x, H)
)
print('CONTINUITY CONDITION FOR THE FLUX AT THE INTERFACE:')
display(continuity_f)
continuity_curr = sp.Eq(
    -D* sp.diff(flux_c.rhs, x).subs(x, H),
    -D * sp.diff(flux_r.rhs, x).subs(x, H)
    )
print('CONTINUITY CONDITION FOR THE CURRENT AT THE INTERFACE:')
display(continuity_curr)

ODE FOR THE CORE:


Eq(B**2*phi_c(x) + Derivative(phi_c(x), (x, 2)), 0)

ODE FOR THE REFLECTOR:


Eq(Derivative(phi_r(x), (x, 2)) - phi_r(x)/L_r**2, 0)

GENERAL SOLUTION FOR THE FLUX IN THE CORE:


...

Eq(phi_r(x), C3*exp(-x/L_r))

CONTINUITY CONDITION FOR THE FLUX AT THE INTERFACE:


Eq(C1*sin(B*H) + C2*cos(B*H), C3*exp(-H/L_r))

CONTINUITY CONDITION FOR THE CURRENT AT THE INTERFACE:


Eq(-D*(B*C1*cos(B*H) - B*C2*sin(B*H)), C3*D*exp(-H/L_r)/L_r)